In [28]:
import requests
from newspaper import Article
import heapq
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from collections import deque
import os
import json
import openpyxl
import pandas as pd
import concurrent.futures
from PyPDF2 import PdfReader
from openpyxl.utils import get_column_letter
import time
from openai import OpenAI
import tiktoken

In [29]:
def extract_text(web_url):
    try:
        if web_url.lower().endswith('.pdf'):
            pdf_filename = os.path.basename(urlparse(web_url).path)

            if not os.path.exists('documents'):
                os.makedirs('documents')

            pdf_filepath = os.path.join('documents', pdf_filename)

            response = requests.get(web_url)
            with open(pdf_filepath, 'wb') as f:
                f.write(response.content)

            try:
                with open(pdf_filepath, 'rb') as f:
                    reader = PdfReader(f)
                    text = ""
                    for page in reader.pages:
                        page_text = page.extract_text()
                        if page_text:
                            text += page_text
                print(f'Done for {web_url}')
               
                return text
            except Exception as e:
                print(f"Failed to read PDF {web_url}: {e}")
                return None  # Return None if PDF reading fails
        else:
            response = requests.get(web_url)
            soup = BeautifulSoup(response.content, 'html.parser')
            text = soup.get_text()
            print(f'Done for {web_url}')
            return text

    except Exception as e:
        print(f'Failed at {web_url} {str(e)}')
        return None 


In [30]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus

query = "latest news on euro usd exchange"
encoded = quote_plus(query)  

# 2) Bing News search URL (not the general web search)
url = f"https://www.bing.com/news/search?q={encoded}"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, "html.parser")

# 3) Grab the news‐card links
for card in soup.select("a.title"):
    title = card.get_text(strip=True)
    link  = card["href"]
    print(f"{link}\n")
    article = Article(link)
    article.download()
    article.parse()
    text = article.text.strip()
    cleaned_text = re.sub(r' {3,}', '  ', text)
    if len(cleaned_text)<50:
        continue
    print("this is it",cleaned_text)


https://www.fxstreet.com/analysis/eur-usd-forecast-euro-loses-momentum-but-remains-bullish-202507230928

this is it Another significant data release for the Euro is the Trade Balance. This indicator measures the difference between what a country earns from its exports and what it spends on imports over a given period. If a country produces highly sought after exports then its currency will gain in value purely from the extra demand created from foreign buyers seeking to purchase these goods. Therefore, a positive net Trade Balance strengthens a currency and vice versa for a negative balance.

Data releases gauge the health of the economy and can impact on the Euro. Indicators such as GDP, Manufacturing and Services PMIs, employment, and consumer sentiment surveys can all influence the direction of the single currency. A strong economy is good for the Euro. Not only does it attract more foreign investment but it may encourage the ECB to put up interest rates, which will directly strengt

ArticleException: Article `download()` failed with 403 Client Error: Forbidden for url: https://in.investing.com/news/forex-news/further-eurusd-declines-could-trigger-large-cta-unwinds-bofa-4919032 on URL https://in.investing.com/news/forex-news/further-eurusd-declines-could-trigger-large-cta-unwinds-bofa-4919032

In [ ]:
YOUR_API_KEY = "sk-6ecb03a05cb642578d94120ee70b780d"
client = OpenAI(api_key="sk-6ecb03a05cb642578d94c20ee70b780d", base_url="https://api.deepseek.com/v1")



In [33]:
for card in soup.select("a.title"):
    title = card.get_text(strip=True)
    link  = card["href"]
    print(f"{link}\n")
    article = Article(link)
    article.download()
    article.parse()
    text = article.text.strip()
    cleaned_text = re.sub(r' {3,}', '  ', text)
    if len(cleaned_text)<50:
        continue
    messages = [
      {
          "role": "system",
          "content": (
              "Be informative"
          ),
      },
      {
          "role": "user",
          "content": (
              f"""\"{cleaned_text}\"\n\nBased on this news, predict whether we should buy EURUSD or sell"""
                     ),
      },
              ]
  
    response = client.chat.completions.create(
          model="deepseek-chat",
          messages=messages,
      )
    response_text = response.choices[0].message.content
    print(response_text)
    break

https://www.fxstreet.com/analysis/eur-usd-forecast-euro-loses-momentum-but-remains-bullish-202507230928

Based on the information provided, here's a breakdown of the key factors influencing **EUR/USD** and a potential trading outlook:

### **Key Factors Supporting EUR/USD (Buy)**
1. **ECB Monetary Policy Decision (Upcoming)**  
   - If the ECB signals a **hawkish stance** (hinting at future rate hikes due to inflation above 2%), the **Euro could strengthen**.  
   - Higher interest rates make the Euro more attractive to investors.

2. **Trade Balance & Economic Data**  
   - A **positive Eurozone trade balance** (strong exports) supports EUR demand.  
   - Strong economic data (GDP, PMIs, employment) from Germany, France, Italy, or Spain could boost the Euro.

3. **Technical Outlook (Bullish Bias)**  
   - The pair recently hit a **two-week high (1.1760)** and has been rising for three consecutive days.  
   - Support levels (1.1700, 1.1650) suggest a **dip-buying opportunity** if the 